# From whole video to multiple pictures all from positive classes.

In [1]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
YOLOV7_FOLDER = '/content/drive/My Drive/Bundesliga/YOLO7'
VIDEOS_FOLDER = '/yolov7/runs/detect'
CLIPS_FOLDER = YOLOV7_FOLDER + VIDEOS_FOLDER
os.chdir(CLIPS_FOLDER)
! pwd

/content/drive/My Drive/Bundesliga/YOLO7/yolov7/runs/detect


### Working with only the first video for the moment

In [4]:
os.chdir(CLIPS_FOLDER + '/clips')
! pwd

/content/drive/My Drive/Bundesliga/YOLO7/yolov7/runs/detect/clips


In [65]:
vidcap = cv2.VideoCapture('08fd33_0.mp4')
success, image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image) # save frame as jpg file      
  success,image = vidcap.read()
  # print('Read a new frame: ', success)
  count += 1

In [6]:
from google.colab.patches import cv2_imshow

### Working with high resolution images for YOLO

In [9]:
os.chdir(CLIPS_FOLDER + '/exp9')
! pwd

/content/drive/My Drive/Bundesliga/YOLO7/yolov7/runs/detect/exp9


In [ ]:
# ! mkdir OriginalFrames

In [16]:
video = '08fd33_0'
video_original = f'{video}_original'
FOLDER_ORIGINAL = 'OriginalFrames/'
FOLDER_LABELS = 'labels/'

Creating frames - not original ones (with YOLO detections):

In [22]:
vidcap = cv2.VideoCapture(f'{video}.mp4')
success, image = vidcap.read()
count = 0
while success:
  cv2.imwrite(f'frame{count}.jpg', image) # save frame as JPEG file      
  success, image = vidcap.read()
  count += 1

Creating frames - original ones (without YOLO detections) to crop:

In [29]:
vidcap = cv2.VideoCapture(f'{video_original}.mp4')
success, image = vidcap.read()
count = 0
while success:
  cv2.imwrite(f'{FOLDER_ORIGINAL}frame{count}_original.jpg', image) # save frame as JPEG file      
  success, image = vidcap.read()
  count += 1

Cropping original frames (without YOLO detections):

In [41]:
CLASS_INDEX = 0
X_INDEX = 1
Y_INDEX = 2
BALL_CLASS = '32'
X_SIZE = 1928
Y_SIZE = 1024
CROP_SIZE = 224 # pixels, size of crop = (CROP_SIZExCROP_SIZE)
radius = int(CROP_SIZE / 2)

In [54]:
from os import listdir
from os.path import isfile, join
files = [file for file in listdir(FOLDER_ORIGINAL) if isfile(join(FOLDER_ORIGINAL, file))] # get a list of files in the directory

In [55]:
filenames = [os.path.splitext(filename)[0] for filename in files] # separate filenames from their extension .jpg

In [64]:
for i, frame in enumerate(filenames, 1): # frame of video (filenames contains frames of a single video)
  label_txt = f'{video}_{i}.txt'
  with open(f'{FOLDER_LABELS}{label_txt}') as f:
    reader = f.read()
  identified_objects = [i.split(' ') for i in reader.split('\n')]

  for object_ in identified_objects:
    if object_[CLASS_INDEX] == BALL_CLASS:
      x = int(float(object_[X_INDEX]) * X_SIZE)
      y = int(float(object_[Y_INDEX]) * Y_SIZE)
      ymin = max(y - radius, 0)
      ymax = min(y + radius, Y_SIZE)
      xmin = max(x - radius, 0)
      xmax = min(x + radius, X_SIZE)

      img = cv2.imread(frame + '.jpg')
      cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
      cv2.imwrite(f'{FOLDER_ORIGINAL}{frame}_cropped.jpg', cropped_image)